In [2]:
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

import pickle
import scipy
import numpy as np
from sklearn import preprocessing, svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, RobustScaler, normalize
from tqdm import tqdm
import pandas as pd


import networkx as nx
from tqdm import tqdm
import random
import numpy as np
import pickle
import os
import glob
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn


seed = 42
random.seed(seed)
np.random.seed(seed)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
with open('../data/hyperlex_pairs_ppl.pickle', 'rb') as f:
    t = pickle.load(f)

# сделаем словарь {пара:перплексия}
ppl_dict = dict()
for elements in t.items():
    v1 = elements[0][0].split('(')[0].strip(' ')
    v2 = elements[0][1].strip(',')
    ppl = elements[1]
    ppl_dict[(v1, v2)] = ppl

In [5]:
train_df = {'forward': [], 'backward': [],  'label': [], 'from': [], 'to': []}


with open('/home/LLM_Taxonomy/LexicalEntailment/data/hyperlex/hyperlex-all.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines()[1:]:
        splitted = line.split(' ')
        v1 = splitted[0]
        v2 = splitted[1]
        score_10 = splitted[5]
        if (v1, v2) in ppl_dict.keys():
            train_df['forward'].append(ppl_dict[(v1, v2)])
            train_df['backward'].append(ppl_dict[(v2, v1)])
            train_df['label'].append(float(score_10))
            train_df['from'].append(v2)
            train_df['to'].append(v1)
        else:
            pass

In [8]:
df = pd.DataFrame(train_df)

In [9]:
df

,forward,backward,label,from,to
0,2.421397,5.009867,8.67,disagreement,conflict
1,8.875535,91.182434,7.50,worker,mason
2,26.051916,38.237873,6.15,light,aura
3,21.442856,7.594214,0.15,carrot,radish
4,4501.588867,701.239014,0.13,duty,tusk
...,...,...,...,...,...
2415,96.594872,7.663406,3.00,climber,person
2416,11.773605,634.021667,6.80,medium,newspaper
2417,26.340752,1627.639282,9.55,event,concert
2418,19189.283203,14.231255,1.53,pie,food


In [27]:
all_verteces = np.hstack([df['from'].values, df['to'].values])

In [28]:
all_verteces

array(['disagreement', 'worker', 'light', ..., 'concert', 'food',
       'avenue'], dtype=object)

In [29]:
len(all_verteces), len(set(all_verteces))

(4840, 2141)

In [30]:
G = nx.DiGraph()
for synset in (wn.all_synsets('n')):
    name = synset.name()
    G.add_node(name)
    hyponyms = synset.hyponyms()

    for hypo in hyponyms:
        new_name = hypo.name()
        G.add_node(new_name)
        G.add_edge(name, new_name)

for synset in (wn.all_synsets('v')):
    name = synset.name()
    G.add_node(name)
    hyponyms = synset.hyponyms()

    for hypo in hyponyms:
        new_name = hypo.name()
        G.add_node(new_name)
        G.add_edge(name, new_name)


In [31]:
all_verteces[:5]

array(['disagreement', 'worker', 'light', 'carrot', 'duty'], dtype=object)

In [38]:
in_wnet = 0
found = []

for node in list(set(all_verteces)):
    node = node.replace(' ', '_')
    for i in range(12):
        true_name = f'{node}.n.0{i}'
        if true_name in G.nodes():
            in_wnet += 1
            found.append(node)
            break
    

In [39]:
node

'decade'

In [40]:
in_wnet

1875

In [41]:
both_found = {}

for idx, elem in df.iterrows():
    child = elem['from']
    parent = elem['to']

    #child = lemmatizer.lemmatize(child, pos='v')
    child = child.replace(' ', '_')

    #parent = lemmatizer.lemmatize(parent, pos='v')
    parent = parent.replace(' ', '_')
    

    children = []
    for i in range(10):
        true_name = f'{child}.v.0{i}'
        if true_name in G.nodes():
            children.append(true_name)
    
    parents = []
    for i in range(10):
        true_name = f'{parent}.v.0{i}'
        if true_name in G.nodes():
            parents.append(true_name)

    both_found[idx] = (children, parents)

In [43]:
drop_keys = []
for k, v in both_found.items():
    if (len(v[0]) == 0) or (len(v[1]) == 0):
        drop_keys.append(k)

In [44]:
for k in drop_keys:
    both_found.pop(k)

In [46]:
len(both_found)

652

In [47]:
paths = []
for k, v in both_found.items():
    children = v[0]
    parents = v[1]

    min_path = 99999999

    for child in children:
        for parent in parents:
            try:
                path = (nx.shortest_path_length(G, parent, child))
            except nx.NetworkXNoPath:
                path = 99999999
            
            if path < min_path:
                min_path = path
    
    paths.append(path)

In [49]:
paths = np.array(paths)

In [50]:
len(paths)

652

In [51]:
(paths != 99999999).sum()

4

In [54]:
paths[(paths != 99999999)].mean()

1.5